## Simeple Baseline - Weather doesn't change

### Extract Test Pictures

The Radar Images from 2016 and 2017 are used to test the Baselines. Therefore the picture get extracted if this wasn't done before.
Before executing this script, the zipped .pngs have to be downloaded and stored in the following structure: 

```
.
├── Data
│   └── TestPicturesZipped
│       ├── 2016
│       │   ├── radar_1601.zip
│       │   ├── radar_1602.zip
│       │   ├── radar_1603.zip
│       │   ├── radar_1604.zip
│       │   ├── radar_1605.zip
│       │   ├── radar_1606.zip
│       │   ├── radar_1607.zip
│       │   ├── radar_1608.zip
│       │   ├── radar_1609.zip
│       │   ├── radar_1610.zip
│       │   ├── radar_1611.zip
│       │   └── radar_1612.zip
│       └── 2017
│           ├── radar_1701.zip
│           ├── radar_1702.zip
│           ├── radar_1703.zip
│           ├── radar_1704.zip
│           ├── radar_1705.zip
│           ├── radar_1706.zip
│           ├── radar_1707.zip
│           ├── radar_1708.zip
│           ├── radar_1709.zip
│           ├── radar_1710.zip
│           ├── radar_1711.zip
│           └── radar_1712.zip
├── README.md
└── SimpleBaseline.ipynb
```

In [2]:
import zipfile
import os
abs_dir_path = os.path.join(os.path.join(os.path.abspath('.')))
data_dir = os.listdir(abs_dir_path + "/Data")

if "TestPictures" not in data_dir:
    print("Pictures get extracted")
    list_years = os.listdir(abs_dir_path + "/Data/TestPicturesZipped")
    list_years.sort()
    for year in list_years:
        list_of_months = os.listdir(abs_dir_path + "/Data/TestPicturesZipped/" + year)
        list_of_months.sort()
        for month in list_of_months:   
            with zipfile.ZipFile(abs_dir_path + "/Data/TestPicturesZipped/" + year + "/" + month) as zip_ref:
                zip_ref.extractall(abs_dir_path + "/Data/TestPictures/" + year + "/" + month.replace(".zip", ""))
else: 
    print("Folder already exists")

Pictures get extracted


In [3]:
print("Path to test pictures get stored")
list_years = os.listdir(abs_dir_path + "/Data/TestPictures")
list_years.sort()
paths_to_test_pics = []
for year in list_years:
    list_of_months = os.listdir(abs_dir_path + "/Data/TestPictures/" + year)
    list_of_months.sort()
    for month in list_of_months:
        path_to_pics_of_month = os.listdir(abs_dir_path + "/Data/TestPictures/" + year + "/" + month)
        path_to_pics_of_month.sort()
        for path in path_to_pics_of_month:
            paths_to_test_pics.append(abs_dir_path + "/Data/TestPictures/" + year + "/" + month + "/" + path)
print(paths_to_test_pics[0]) 
print(len(paths_to_test_pics))

Path to test pictures get stored
/home/paul/Documents/Master1/DeepRain_Teamproject/DeepRain2/Data/TestPictures/2016/radar_1601/scaled_1601010000.png
210528


In [4]:
def get_pixel_of_constance(pic_path):
    pic = io.imread(pic_path)[820:1020, 400:600]
    pic_transformed = transform.resize(pic, output_shape = (64,64))
    pixel_of_constance = pic_transformed[posx, posy]
    return pixel_of_constance

In [5]:
from skimage import io, transform
import numpy as np
import pickle
import time

data_dir = os.listdir(abs_dir_path)

if "pixel_of_constance2016_17.p" not in data_dir:
    #Postition of Constance: X 31, Y 40
    posx, posy = 31, 40,
    counter = 1
    pixel_of_constance = []
    start_time = time.time()
    for pic_path in paths_to_test_pics:
        pixel_of_constance.append([pic_path, get_pixel_of_constance(pic_path)]) 
        if paths_to_test_pics.index(pic_path)%1000 == 0 and paths_to_test_pics.index(pic_path) != 0:
            print(str(counter*1000) + " done")
            print("--- %s seconds ---" % (time.time() - start_time))
            start_time = time.time()
            counter +=1
            break

    pickle.dump(pixel_of_constance, open("pixel_of_constance2016_17.p", "wb"))
else: 
    print("File already exists")

1000 done
--- 22.42480754852295 seconds ---


In [6]:
pixel_of_constance1617 = pickle.load(open("pixel_of_constance2016_17.p", "rb"))
print("Length of List: " + str(len(pixel_of_constance1617)))
pixel_of_constance1617[0]

Length of List: 1001


['/home/paul/Documents/Master1/DeepRain_Teamproject/DeepRain2/Data/TestPictures/2016/radar_1601/scaled_1601010000.png',
 0.0]

In [152]:
import time
start_time = time.time()

forecast_in_5_min_steps = 5   #min. 5 min
forecast_pic_distance = int(forecast_in_5_min_steps/5)

forecasts = []
#for index in range(len(paths_to_test_pics)-(forecast_pic_distance)):
for index in range(2000-(forecast_pic_distance)):
    first_pic_path = paths_to_test_pics[index]
    second_pic_path = paths_to_test_pics[index+forecast_pic_distance]
    
    #get Constance Pixelvalue and make a Prediction
    first_pixel_value = get_pixel_of_constance(first_pic_path)
    if first_pixel_value == 0:
        #print("It will not rain")
        prediction = 0
    elif first_pixel_value < mean_rain_intensity/2:
        #print("It will rain a little bit")
        prediction = 1
    else:
        #print("It will rain")
        prediction = 2
    
        
    # Compare with actuale weather
    second_pixel_value = get_pixel_of_constance(second_pic_path)
    if second_pixel_value == 0:
        #print("no rain")
        actual_weather = 0
    elif second_pixel_value < mean_rain_intensity/2:
        #print("little rain")
        actual_weather = 1
    else:
        #print("rain")
        actual_weather = 2
    forecasts.append([prediction, actual_weather])

print("--- %s seconds ---" % (time.time() - start_time))

forecast_hitrate = []
z = 0
a = 0
for i in range(len(forecasts)):
    if forecasts[i][0] == forecasts[i][1]:
        z += 1
    else: 
        a += 1
ergebnis = z/(a+z)
print(ergebnis)

--- 31.782063484191895 seconds ---
1.0


In [119]:
a = [["test",1],["hallo", 2]]
np.asanyarray(a)[:,1]

array(['1', '2'], dtype='<U5')